In [ ]:
from IPython.core.display import HTML
from IPython.display import display, clear_output

display(HTML("<style>.output_subarea { overflow: auto; }</style>"))

import datetime
import random
import os
import pandas as pd
import socket

from supabase import create_client, Client
from ib_async import *

import urllib

hostname = socket.gethostname()
util.logToFile(f"{datetime.datetime.now().strftime('%Y-%m-%d')}-{hostname}.log")
util.startLoop()
util.logToConsole()

if "ib" in globals():
    ib.disconnect()

randint = lambda a=1, b=10: random.randint(a, b)
ib = IB()
ib.connect("127.0.0.1", 4001, randint(1, 99))

import platform

if platform.system() == "Linux":
    import chime


def chime_success():
    if platform.system() == "Linux":
        chime.success()


chime_success()
os.system("say beep")

# setup listeners for ib
# NQM2024 contract
print("Setting contract to NQM2024 symbol / JUN 2024 / Contract(conId=620730920)")
contract = Contract(conId=620730920)
ib.qualifyContracts(contract)

ticker = ib.reqMktDepth(contract)

ib.reqPositions()
ib.reqAccountSummary()
allopenorders = ib.reqAllOpenOrders()

executions = ib.reqExecutions()

ib.accountSummary()
ib.accountValues()
ib.accountValueEvent()


def push_notifications(msg="Hello world!"):
    try:
        body = f"[{datetime.datetime.now()}] {msg}"
        print(body)
        data = urllib.parse.urlencode({"text": body}).encode()
        req = urllib.request.Request(
            "https://api.chanify.net/v1/sender/CICswLUGEiJBQUZIR0pJQ0VVNkxUTlZCMk1DRElCWU1RSlNWMktCS0NFIgIIAQ.vj8gcfxM4jD9Zv0mBMSlFlY51EL_jC5dB8LWdWX1tAs",
            data=data,
        )
        response = urllib.request.urlopen(req)
        response.read()  # Read the response to ensure the request is complete
    except urllib.error.URLError as e:
        print(f"Error sending request: {e.reason}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


def print_reqOpenOrders():
    print("Session Orders::")
    display(util.df([t.order for t in ib.reqOpenOrders()]))


def print_reqAllOpenOrders():
    print("All Session Orders:")
    display(util.df([t.order for t in ib.reqAllOpenOrders()]))


def print_account_summary(ib):
    print("ACCOUNT SUMMARY::\n")
    acct_fields = ib.accountSummary(account="U10394496")

    for f in acct_fields:
        if "DayTrades" not in f.tag:
            print(f.tag, ":", f.value)
    print()


def print_order(o):
    if o is None:
        print()
        return

    order = o.order
    contract = o.contract
    orderStatus = o.orderStatus

    print(f"symbol\tpermId\t\tstatus\t\taction\tfilled\tremaining\tlmtPrice")

    print(
        f"{contract.symbol}\t{order.permId}\t{orderStatus.status}\t{order.action}\t{orderStatus.filled}\t{orderStatus.remaining}\t\t{order.lmtPrice}\t"
    )


def print_orderbook():
    print("NQ Order Book::")
    if ticker is not None and ticker.domBids is not None and ticker.domAsks is not None:
        for i in range(min(len(ticker.domBids), len(ticker.domAsks))):
            bid_size = ticker.domBids[i].size
            bid_price = ticker.domBids[i].price
            ask_price = ticker.domAsks[i].price
            ask_size = ticker.domAsks[i].size
            print(f"{bid_size:>8} {bid_price:>10} | {ask_price:<10} {ask_size:<8}")

    print()


def print_executions():
    fills = [t.fills for t in ib.trades() if t.fills != []]
    executions = [f[0].execution for f in fills]
    print(f"Intraday Executions: {len(executions)}\n")


def print_positions():
    print("POSITIONS::")
    future = [pos for pos in ib.positions()]
    for f in future:
        print(
            f"{f.contract.symbol} {f.position} @ {f.avgCost/float(contract.multiplier)}"
        )
    print()


def print_summary():
    print_orderbook()
    print_executions()

    print_positions()
    print_account_summary(ib=ib)


def print_openOrders(status_list=["Submitted", "PendingSubmit", "PendingCancel"]):
    print(":: openOrders ::")
    display(util.df([t for t in ib.openOrders()]))


def print_submittedtrades():
    print(":: trades ::")
    display(
        util.df(
            [
                t.order
                for t in ib.trades()
                if t.orderStatus.status == "Submitted"
                or t.orderStatus.status == "PreSubmitted"
                or t.orderStatus.status == "PendingSubmit"
            ]
        )
    )


print_summary()

print_submittedtrades()


2024-05-23 15:29:02,627 ib_async.ib INFO Disconnecting from 127.0.0.1:4001, 882 B sent in 15 messages, 3.44 MB received in 116607 messages, session time 1.93 ks.
2024-05-23 15:29:02,632 ib_async.client INFO Disconnecting
2024-05-23 15:29:02,639 ib_async.client INFO Disconnected.
2024-05-23 15:29:02,648 ib_async.client INFO Connecting to 127.0.0.1:4001 with clientId 83...
2024-05-23 15:29:02,655 ib_async.client INFO Connected


2024-05-23 15:29:02,677 ib_async.client INFO Logged on to server version 176
2024-05-23 15:29:02,687 ib_async.wrapper INFO Warning 2104, reqId -1: Market data farm connection is OK:usfuture
2024-05-23 15:29:02,689 ib_async.wrapper INFO Warning 2104, reqId -1: Market data farm connection is OK:usfarm
2024-05-23 15:29:02,692 ib_async.wrapper INFO Warning 2107, reqId -1: HMDS data farm connection is inactive but should be available upon demand.ushmds
2024-05-23 15:29:02,695 ib_async.wrapper INFO Warning 2158, reqId -1: Sec-def data farm connection is OK:secdefil
2024-05-23 15:29:02,697 ib_async.client INFO API connection ready
2024-05-23 15:29:02,711 ib_async.wrapper INFO position: Position(account='U2340948', contract=Stock(conId=4762, symbol='BA', exchange='NYSE', currency='USD', localSymbol='BA', tradingClass='BA'), position=254.0, avgCost=203.21112205)
2024-05-23 15:29:02,749 ib_async.wrapper INFO position: Position(account='U10394496', contract=Option(conId=637079300, symbol='HYG', l

Setting contract to NQM2024 symbol / JUN 2024 / Contract(conId=620730920)


2024-05-23 15:29:04,737 ib_async.wrapper INFO openOrder: Trade(contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', right='?', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ'), order=Order(orderId=311, clientId=54, permId=1735429010, action='SELL', totalQuantity=1.0, orderType='LMT', lmtPrice=18766.75, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(orderId=311, status='Submitted', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[], advancedError='')
2024-05-23 15:29:04,740 ib_async.wrapper INFO orderStatus: Trade(contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContra

NQ Order Book::
     2.0    18689.5 | 18690.0    5.0     
     3.0   18689.25 | 18690.25   5.0     
     1.0    18689.0 | 18690.5    2.0     
     5.0   18688.75 | 18690.75   6.0     
     4.0    18688.5 | 18691.0    6.0     

Intraday Executions: 176

POSITIONS::
BA 254.0 @ 10.1605561025
EDV 650.0 @ 3.885403845
RSX 100.0 @ 0.34449885
HYG 50.0 @ 0.6223425
AAPL 4.0 @ 9.109842500000001
QQQ 1.0 @ 25.051845
NVDA 2.0 @ 17.471155
GOOGL 10.0 @ 2.5723425
NVDA 1.0 @ 29.652342499999996
PINS 40.0 @ 0.6723425
NQ 1.0 @ 18683.7390166775
NVDA 1.0 @ 25.051845
QQQ 2.0 @ 10.384842500000001
AMZN 4.0 @ 5.7843425

ACCOUNT SUMMARY::

AccountType : INDIVIDUAL
Cushion : 0.769435
LookAheadNextChange : 1716557400
AccruedCash : 7.70
AvailableFunds : 47781.78
BuyingPower : 312069.62
EquityWithLoanValue : 78017.40
ExcessLiquidity : 78017.40
FullAvailableFunds : 47781.78
FullExcessLiquidity : 78017.40
FullInitMarginReq : 23087.22
FullMaintMarginReq : 20988.38
GrossPositionValue : 2382.25
InitMarginReq : 23087.22
Lo

,orderId,clientId,permId,action,totalQuantity,orderType,lmtPrice,auxPrice,tif,activeStartTime,...,usePriceMgmtAlgo,duration,postToAts,advancedErrorOverride,manualOrderTime,minTradeQty,minCompeteSize,competeAgainstBestOffset,midOffsetAtWhole,midOffsetAtHalf
0,311,54,1735429010,SELL,1.0,LMT,18766.75,0.0,GTC,,...,False,2147483647,2147483647,,,2147483647,2147483647,1.797693e+308,1.797693e+308,1.797693e+308
1,378,6,1735430610,BUY,1.0,LMT,18670.25,0.0,GTC,,...,False,2147483647,2147483647,,,2147483647,2147483647,1.797693e+308,1.797693e+308,1.797693e+308
2,379,6,1735430611,BUY,1.0,LMT,18670.25,0.0,GTC,,...,False,2147483647,2147483647,,,2147483647,2147483647,1.797693e+308,1.797693e+308,1.797693e+308
3,0,0,2099801404,BUY,1.0,LMT,18040.50,0.0,GTC,,...,False,2147483647,2147483647,,,2147483647,2147483647,1.797693e+308,1.797693e+308,1.797693e+308
4,271,69,196158554,SELL,1.0,LMT,18924.50,0.0,GTC,,...,False,2147483647,2147483647,,,2147483647,2147483647,1.797693e+308,1.797693e+308,1.797693e+308
5,401,40,1735431452,BUY,1.0,LMT,18685.25,0.0,GTC,,...,False,2147483647,2147483647,,,2147483647,2147483647,1.797693e+308,1.797693e+308,1.797693e+308
6,196,69,196158479,SELL,1.0,LMT,19021.75,0.0,GTC,,...,False,2147483647,2147483647,,,2147483647,2147483647,1.797693e+308,1.797693e+308,1.797693e+308


2024-05-23 15:39:55,209 ib_async.wrapper INFO position: Position(account='U10394496', contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', currency='USD', localSymbol='NQM4', tradingClass='NQ'), position=2.0, avgCost=373689.890166775)
2024-05-23 15:39:55,218 ib_async.wrapper INFO position: Position(account='U10394496', contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', currency='USD', localSymbol='NQM4', tradingClass='NQ'), position=2.0, avgCost=373690.890166775)
2024-05-23 15:44:39,940 ib_async.wrapper INFO position: Position(account='U10394496', contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', currency='USD', localSymbol='NQM4', tradingClass='NQ'), position=2.0, avgCost=373690.89016675)
2024-05-23 15:59:18,490 ib_async.wrapper ERROR Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
2024-05-23 1

In [66]:
def parse_ibrecords(data_array):

    data_list = []

    for obj in data_array:
        data = {}

        if hasattr(obj, "contract"):
            util.logging.debug(obj.contract)
            contract = util.dataclassNonDefaults(obj.contract)
            data = {**data, **contract}

        if hasattr(obj, "order"):
            util.logging.debug(obj.order)
            order = util.dataclassNonDefaults(obj.order)
            order.pop("softDollarTier")
            data = {**data, **order}

        if hasattr(obj, "orderStatus"):
            util.logging.debug(obj.orderStatus)
            orderStatus = util.dataclassNonDefaults(obj.orderStatus)
            data = {**data, **orderStatus}

        if hasattr(obj, "fills"):
            util.logging.debug(obj.fills)
            fills = {"fills": obj.fills}
            data = {**data, **fills}

        if hasattr(obj, "log"):
            util.logging.debug(obj.log)
            logs = {"log": [util.dataclassAsDict(e) for e in obj.log]}
            data = {**data, **logs}

        if hasattr(obj, "advancedError"):
            util.logging.debug(obj.advancedError)
            advancedError = {"advancedError": obj.advancedError}
            data = {**data, **advancedError}

        if type(obj) == Order:
            util.logging.debug(obj)
            order = util.dataclassNonDefaults(obj)
            order.pop("softDollarTier")
            data = {**data, **order}

        data_list.append(data)

    return data_list


def print_ibrecords_table(
    data_array,
    cols=[
        "localSymbol",
        "permId",
        "status",
        "orderType",
        "action",
        "lmtPrice",
        "remaining",
    ],
):
    df = pd.DataFrame(data_array)
    df = df[cols]
    print(df)


# openTrades = parse_ibrecords(ib.openTrades())
accountSummary = ib.accountSummary()
accountValues = ib.accountValues()

openTrades = parse_ibrecords(ib.openTrades())
# util.df([d for d in openTrades])

openOrders = parse_ibrecords(ib.openOrders())
# util.df([d for d in openOrders])

trades = parse_ibrecords(ib.trades())
# util.df([d for d in trades])

orders = parse_ibrecords(ib.orders())
# util.df([d for d in orders])


In [67]:
openTrades


[{'secType': 'FUT',
  'conId': 620730920,
  'symbol': 'NQ',
  'lastTradeDateOrContractMonth': '20240621',
  'right': '?',
  'multiplier': '20',
  'exchange': 'CME',
  'currency': 'USD',
  'localSymbol': 'NQM4',
  'tradingClass': 'NQ',
  'orderId': 311,
  'clientId': 54,
  'permId': 1735429010,
  'action': 'SELL',
  'totalQuantity': 1.0,
  'orderType': 'LMT',
  'lmtPrice': 18766.75,
  'auxPrice': 0.0,
  'tif': 'GTC',
  'ocaType': 3,
  'displaySize': 2147483647,
  'rule80A': '0',
  'openClose': '',
  'volatilityType': 0,
  'deltaNeutralOrderType': 'None',
  'referencePriceType': 0,
  'account': 'U10394496',
  'clearingIntent': 'IB',
  'adjustedOrderType': 'None',
  'cashQty': 0.0,
  'dontUseAutoPriceForHedge': True,
  'status': 'Submitted',
  'remaining': 1.0,
  'fills': [],
  'log': [{'time': datetime.datetime(2024, 5, 23, 19, 56, 52, 987711, tzinfo=datetime.timezone.utc),
    'status': 'Submitted',
    'message': '',
    'errorCode': 0}],
  'advancedError': ''},
 {'secType': 'FUT',
  '

In [64]:
print("POSITIONS::")
future = [pos for pos in ib.positions() if pos.contract.secType == "FUT"]
for f in future:
    print(f"{f.contract.symbol} {f.position} @ {f.avgCost/float(contract.multiplier)}")
print()


POSITIONS::
NQ 3.0 @ 18665.046700696672



In [ ]:
# print multiple columns of a dictionary
{k: openTrades[0][k] for k in ["symbol", "conId", "localSymbol", "secType"]}


In [53]:
temp


{'secType': 'FUT',
 'conId': 620730920,
 'symbol': 'NQ',
 'lastTradeDateOrContractMonth': '20240621',
 'right': '?',
 'multiplier': '20',
 'exchange': 'CME',
 'currency': 'USD',
 'localSymbol': 'NQM4',
 'tradingClass': 'NQ',
 'orderId': 311,
 'clientId': 54,
 'permId': 1735429010,
 'action': 'SELL',
 'totalQuantity': 1.0,
 'orderType': 'LMT',
 'lmtPrice': 18766.75,
 'auxPrice': 0.0,
 'tif': 'GTC',
 'ocaType': 3,
 'displaySize': 2147483647,
 'trailStopPrice': 18767.75,
 'volatilityType': 0,
 'deltaNeutralOrderType': 'None',
 'referencePriceType': 0,
 'account': 'U10394496',
 'clearingIntent': 'IB',
 'adjustedOrderType': 'None',
 'cashQty': 0.0,
 'dontUseAutoPriceForHedge': True,
 'status': 'Submitted',
 'remaining': 1.0,
 'fills': [],
 'log': [{'time': datetime.datetime(2024, 5, 23, 18, 55, 52, 636184, tzinfo=datetime.timezone.utc),
   'status': 'Submitted',
   'message': '',
   'errorCode': 0}],
 'advancedError': ''}

In [ ]:
print(
    util.df(ib.openOrders())
    .sort_values("lmtPrice", ascending=False)
    .loc[
        :,
        [
            "orderId",
            "permId",
            "action",
            "totalQuantity",
            "orderType",
            "lmtPrice",
            "tif",
        ],
    ]
)


In [ ]:
util.df(ib.positions()).sort_values("account", ascending=False)


In [ ]:
p


In [ ]:
openOrders[0]


In [ ]:
openOrders = ib.openOrders()
util.logging.debug(
    util.df(openOrders)
    .sort_values("lmtPrice", ascending=False)
    .loc[
        :,
        [
            "orderId",
            "permId",
            "action",
            "totalQuantity",
            "orderType",
            "lmtPrice",
            "tif",
        ],
    ]
)


In [ ]:
util.df([t.orderStatus for t in ib.reqAllOpenOrders()])


In [ ]:
df = util.df(
    [
        t.order
        for t in ib.openTrades()
        if t.orderStatus.status == "Submitted"
        or t.orderStatus.status == "PreSubmitted"
        or t.orderStatus.status == "PendingSubmit"
    ]
)


In [ ]:
df[df["permId"] == 1394515158]


In [ ]:
def print_openorders():
    orders = ib.openOrders()
    orders.sort(key=lambda order: order.lmtPrice)

    # Order(orderId=658, clientId=3124, permId=342738244, action='SELL', totalQuantity=1.0, orderType='LMT', lmtPrice=17692.5, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True)

    util.logging.debug(f"orderId\tpermId\t\taction\ttotalQuantity\tlmtPrice\ttif")
    for order in orders:
        util.logging.debug(
            f"{order.orderId}\t{order.permId}\t{order.orderType}\t{order.action}\t{order.totalQuantity}\t{order.lmtPrice} \t{order.tif}"
        )
    util.logging.debug()


print_openorders()


In [ ]:
openOrders = ib.openOrders()
openTrades = ib.openTrades()


In [ ]:
obj = openTrades[0]
if hasattr(obj, "contract"):
    util.logging.debug(obj.contract)

if hasattr(obj, "order"):
    util.logging.debug(obj.order)

if hasattr(obj, "orderStatus"):
    util.logging.debug(obj.orderStatus)

if hasattr(obj, "fills"):
    util.logging.debug(obj.fills)

if hasattr(obj, "log"):
    util.logging.debug(obj.log)

if hasattr(obj, "advancedError"):
    util.logging.debug(obj.advancedError)


In [ ]:
def parse_ibrecords(data_array):

    data_list = []

    for obj in data_array:
        data = {}

        if hasattr(obj, "contract"):
            util.logging.debug(obj.contract)
            contract = util.dataclassNonDefaults(obj.contract)
            data = {**data, **contract}

        if hasattr(obj, "order"):
            util.logging.debug(obj.order)
            order = util.dataclassNonDefaults(obj.order)
            order.pop("softDollarTier")
            data = {**data, **order}

        if hasattr(obj, "orderStatus"):
            util.logging.debug(obj.orderStatus)
            orderStatus = util.dataclassNonDefaults(obj.orderStatus)
            data = {**data, **orderStatus}

        if hasattr(obj, "fills"):
            util.logging.debug(obj.fills)
            fills = {"fills": obj.fills}
            data = {**data, **fills}

        if hasattr(obj, "log"):
            util.logging.debug(obj.log)
            logs = {"log": [util.dataclassAsDict(e) for e in obj.log]}
            data = {**data, **logs}

        if hasattr(obj, "advancedError"):
            util.logging.debug(obj.advancedError)
            advancedError = {"advancedError": obj.advancedError}
            data = {**data, **advancedError}

        util.logging.debug(f"Processed orderId :: {order['permId']}")

        data_list.append(data)

    return data_list


parse_ibrecords([openTrades[0]])


In [ ]:
import sqlite3
from datetime import datetime
from typing import Dict, List, Any, Optional


class SQLiteWrapper:
    def __init__(self, db_name: str, drop_tables=[]):
        self.conn = sqlite3.connect(db_name)
        self.drop_tables(drop_tables)
        self.create_trades_table()

    def drop_tables(self, drop_tables: List[str]):
        for tbl in drop_tables:
            drop_table_sql = f"DROP TABLE IF EXISTS {tbl};"
            self.conn.execute(drop_table_sql)
            self.conn.commit()

    def create_trades_table(self):

        create_table_sql = """
        CREATE TABLE IF NOT EXISTS trades (
            secType TEXT,
            conId INTEGER,
            symbol TEXT,
            lastTradeDateOrContractMonth TEXT,
            right TEXT,
            multiplier TEXT,
            exchange TEXT,
            currency TEXT,
            localSymbol TEXT,
            tradingClass TEXT,
            orderId INTEGER,
            clientId INTEGER,
            permId INTEGER PRIMARY KEY,
            action TEXT,
            totalQuantity REAL,
            filledQuantity REAL,
            orderType TEXT,
            lmtPrice REAL,
            auxPrice REAL,
            trailStopPrice REAL,
            strike REAL,
            tif TEXT,
            parentPermId INTEGER,
            ocaType INTEGER,
            displaySize INTEGER,
            rule80A TEXT,
            openClose TEXT,
            volatilityType INTEGER,
            deltaNeutralOrderType TEXT,
            shareholder TEXT,
            referencePriceType INTEGER,
            refFuturesConId INTEGER,
            account TEXT,
            clearingIntent TEXT,
            autoCancelDate TEXT,
            adjustedOrderType TEXT,
            cashQty REAL,
            dontUseAutoPriceForHedge INTEGER,
            fills TEXT,
            log TEXT,
            advancedError TEXT,
            status TEXT,
            remaining REAL,
            ocaGroup TEXT
        );
        """
        self.conn.execute(create_table_sql)
        self.conn.commit()

    def upsert_trades_record(self, data: Dict[str, Any]):
        # Convert datetime objects in log to string
        log_entries = data.get("log", [])
        for entry in log_entries:
            entry["time"] = entry["time"].isoformat()
        data["log"] = str(log_entries)

        # Convert fills list to string
        data["fills"] = str(data.get("fills", []))

        # Handle boolean conversion
        data["dontUseAutoPriceForHedge"] = int(data["dontUseAutoPriceForHedge"])

        keys = ", ".join(data.keys())
        question_marks = ", ".join("?" for _ in data)
        update_clause = ", ".join(f"{key}=excluded.{key}" for key in data)

        upsert_sql = f"""
        INSERT INTO trades ({keys})
        VALUES ({question_marks})
        ON CONFLICT(permId) DO UPDATE SET
        {update_clause};
        """

        values = tuple(data.values())
        self.conn.execute(upsert_sql, values)
        self.conn.commit()

    def read_record(self, permId: int) -> Optional[Dict[str, Any]]:
        select_sql = "SELECT * FROM orders WHERE permId = ?;"
        cursor = self.conn.execute(select_sql, (permId,))
        row = cursor.fetchone()
        if row:
            col_names = [description[0] for description in cursor.description]
            record = dict(zip(col_names, row))
            # Convert log string back to list of dicts
            record["log"] = eval(record["log"])
            # Convert fills string back to list
            record["fills"] = eval(record["fills"])
            # Convert integer back to boolean
            record["dontUseAutoPriceForHedge"] = bool(
                record["dontUseAutoPriceForHedge"]
            )
            return record
        return None

    def close(self):
        self.conn.close()


def parse_OpenTrades(ib: IB):
    trades = ib.trades()

    data = None
    data_list = []

    for trade in trades:
        contract = util.dataclassNonDefaults(trade.contract)
        order = util.dataclassNonDefaults(trade.order)
        order.pop("softDollarTier")
        orderStatus = util.dataclassNonDefaults(trade.orderStatus)
        order["fills"] = trade.fills
        order["log"] = [util.dataclassAsDict(e) for e in trade.log]
        order["advancedError"] = trade.advancedError

        util.logging.debug(f"Processing trade with orderId: {order['permId']}")

        data = {**contract, **order, **orderStatus}

        data_list.append(data)

    return data_list


def insert_OpenTrades(ib: IB, db: SQLiteWrapper):
    data_list = parse_OpenTrades(ib)

    for data in data_list:
        try:
            db.upsert_trades_record(data)

        except Exception as e:
            util.logging.debug(f"Error upserting record: {e}")
            util.logging.debug(data)

    return True


def test_trades_upsert():
    # db = SQLiteWrapper('test.db', drop_tables = ['trades', 'orders'])
    db = SQLiteWrapper("test.db", drop_tables=["trades"])

    insert_OpenTrades(ib, db)

    db.close()

    return True


def test_read_record():
    db = SQLiteWrapper("test.db")
    record = db.read_record(262255346)
    util.logging.debug(record)
    db.close()
    return True


# Example usage
if __name__ == "__main__":
    test_trades_upsert()
